In [1]:
import numpy as np
import sqlite3
from scipy.spatial import distance #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html


In [2]:
conn = sqlite3.connect('../livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select * from livs where "Huvudgrupp" = "Grönsaker"')
for row in rows:
        result.append(row)

db_contents = np.array(result)

conn.close()

## Prototypiska i sin kategori
Syftet är att ta en huvudgrupp och se vad mittpunkten i den kategorin är och sen räkna ut vilka livsmedel som ligger närmast mittpunkten.

Varje näringsvärde blir en dimension, och varje livsmedel en punkt som har ett värde i alla de dimensionerna.

Medelvärdet av alla livsmedel blir en punkt som man kan utgå från när man räknar avstånd till alla livsmedlen, och när man har tillgång till avstånden så kan man ordna alla livsmedlen efter avståndet till mittpunkten.

Det är första steget. 

Andra steget är att se om man verkligen behöver alla näringsvärden för att ändå få samma ordning mellan livsmedlen. Vilka kan man ta bort?

### Utgångspunkter

Från Livsmedelsverkets data har vi valt en huvudgrupp, som "Grönsaker". Alla dess data ligger i db_contents, och den variabeln har en shape som säger hur många rader (livsmedel) och kolumner (namn, nummer, näringsvärden, klassificering) den innehåller.

In [3]:
db_contents.shape

(94, 63)

Kolumn 0 är namnet, 1 är numret och sen börjar näringsvärdena. 

columns är variabeln som innehåller de värden som används i beräkningarna. Under står de kolumner som är möjliga att använda. 

Vill man ha fram betydelsena av kolumnerna kan man göra något i stil med

```
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
``` 

In [53]:
columns = np.array([ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])
# Possible columns: [ 2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 37, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57]


dataset blir de kolumner som används. Från databasen brukar de komma som unicode, så det är lämpligt att konvertera till float.

In [54]:
dataset = db_contents[:,columns].astype(float)

Det är fantastiskt enkelt att räkna medelvärdet av varje kolumn med funktionen numpy.mean, som returnerar en array med lika många element som antal kolumner

Obs: axis behövs, för annars får man fel medelvärde...

In [55]:
means = np.mean(dataset, axis=0)

In [56]:
print(means, '\n', means.shape)

[  4.23425532e+01   5.17563830e+00   1.07510638e+00   1.88117021e+00
   2.17787234e+00   8.85922340e+01   0.00000000e+00   1.10702128e+00
   1.91085106e+00   9.59574468e-01   9.27659574e-01   5.31914894e+00
   1.81063830e-01   5.85106383e-03   8.51063830e-04] 
 (15,)


Det är läskigt lätt även att räkna avståndet mellan alla punkterna i dataset och medelvärdena:

(Det är något med medelvärdena som gör att man måste sätta en extra klammer runt, för att dimensionaliteten ska stämma...)

In [57]:
dist = distance.cdist(dataset, [means], 'euclidean') #https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html

*dist* innehåller sen varje rads avstånd från medelvärdet i alla kolumner.

In [58]:
dist

array([[ 157.26399283],
       [  30.50805558],
       [  19.26456795],
       [  20.05802698],
       [   9.44167616],
       [  19.88776668],
       [   6.64565365],
       [   6.69804034],
       [  28.85219649],
       [  30.28456786],
       [  15.40097323],
       [   6.64443518],
       [  14.33488082],
       [  31.04100558],
       [  29.62093419],
       [  12.58746631],
       [  14.53411466],
       [   6.92737706],
       [ 128.52198606],
       [ 119.4367159 ],
       [ 111.87940644],
       [  22.55329088],
       [  14.29881678],
       [  25.25485084],
       [  21.54938899],
       [   9.41342453],
       [  17.50778823],
       [  14.28737107],
       [  12.62080921],
       [  29.71246439],
       [  15.32965592],
       [  23.17018801],
       [  26.46566543],
       [  23.99838695],
       [  29.30980919],
       [  26.36861303],
       [   7.80125656],
       [  18.12678322],
       [  14.57101574],
       [ 123.98144987],
       [  24.8203815 ],
       [  16.963

Jag har svårt att vänja mig vid det här med implicit sortering.

Först sorterar vi alltså värdena och får en "ordning", som vi sen använder som index för den array vi egentligen vill sortera...

In [59]:
order = np.argsort(dist.T[0])

In [60]:
order

array([76, 11,  6,  7, 93, 17, 66, 36, 90, 45, 25,  4, 64, 77, 15, 28, 65,
       27, 22, 68, 12, 16, 38, 70, 48, 30, 10, 81, 67, 83, 43, 69, 41, 42,
       26, 37, 47, 86, 85, 44,  2, 62, 71, 73, 87,  5,  3, 51, 46, 60, 24,
       50, 92, 21, 31, 33, 40, 84, 23, 57, 35, 61, 32, 56, 63, 82, 88, 49,
       55,  8, 75, 34, 14, 29,  9,  1, 13, 72, 89, 74, 79, 58, 59, 53, 91,
       54, 52, 20, 78, 19, 39, 18, 80,  0])

Eftersom vi har vår ursprungliga lista med oss i *db_contents* så kan vi skriva ut vilka kolumner som helst, och nu med raderna i rätt ordning utifrån medelvärdet i huvudgruppen.

Här tog det ett tag för mig att förstå hur jag skulle se på resultatet. Att se de första som näringsmässigt "lika" var inte så svårt, men hur är det med de livsmedel som hamnar sist i listan?

Jag tror att man ska se det som en (mångdimensionell) "periferi" där de sista i listan förstås är långt från centrum, men inte alls inbördes "lika". De kan ha stort avstånd till mitten i väldigt skilda dimensioner. Det enda som förenar dem är att de är udda... :-)

In [61]:
db_contents[order,0]

array(['Ättiksgurka u lag', 'Grönkål', 'Brysselkål', 'Brysselkål fryst',
       'Brysselkål kokt', 'Lök gul', 'Lök kokt', 'Trädgårdskrasse',
       'Lök röd', 'Chilipeppar färsk', 'Persilja blad', 'Broccoli',
       'Gurka färskinlagd', 'Kronärtskocka kokt', 'Kronärtskocka',
       'Rödkål', 'Lök stekt', 'Purjolök', 'Nässlor förvällda',
       'Purjolök kokt', 'Grönkål fryst', 'Kålrabbi', 'Vitkål',
       'Vitkål kokt', 'Lök gul fryst', 'Savojkål', 'Fänkål', 'Fänkål kokt',
       'Paprika förvälld', 'Rucolasallat', 'Gräslök', 'Spenat fräst',
       'Sparris grön el vit', 'Dill', 'Pumpa', 'Vattenkrasse',
       'Paprika grön röd fryst', 'Broccoli kokt',
       'Småtomater röda typ körsbärstomat', 'Basilika färsk', 'Blomkål',
       'Blomkål kokt', 'Aubergine kokt', 'Gurka inlagd', 'Spenat färsk',
       'Broccoli fryst', 'Blomkål fryst', 'Bambuskott konserv u lag',
       'Paprika gul', 'Tomater krossade konserv m lag', 'Paprika röd',
       'Paprika grön gul röd', 'Sparris grön kokt', 

## Del två. Att jämföra ordningar...

Den här ordningen mellan livsmedel bygger på att man tar ett urval kolumner. Typiskt tar man så mycket information man har (= alla kolumner) för att få en mer relevant ordning.

Men kanske är det så att inte alla kolumner behövs för att ge den här ordningen? Vilka kolumner skulle man kunna ta bort?

Det enklaste är att kolla om ordningarna är identiska, så i nedanstående är det bara det som görs. 

Dags att summera ihop mycket av del 1 i en funktion. Utgå från db_contents som typ en global variabel. Skicka in kolumner till funktionen, så returneras en ordning mellan livsmedlen i db_contents/dataset

In [62]:
def get_order(columns):
    dataset = db_contents[:,columns].astype(float)
    means = np.mean(dataset, axis=0)
    dist = distance.cdist(dataset, [means], 'euclidean')
    order = np.argsort(dist.T[0])
    return order

Vi ska jämföra ursprungsordningen (order_0) med alla ordningar som uppkommer när man tar bort en kolumn. Vi kallar underordningarna för order_1 och löper igenom dem en i taget.

In [63]:
columns_0 = columns
order_0 = get_order(columns)

In [64]:
order_0

array([76, 11,  6,  7, 93, 17, 66, 36, 90, 45, 25,  4, 64, 77, 15, 28, 65,
       27, 22, 68, 12, 16, 38, 70, 48, 30, 10, 81, 67, 83, 43, 69, 41, 42,
       26, 37, 47, 86, 85, 44,  2, 62, 71, 73, 87,  5,  3, 51, 46, 60, 24,
       50, 92, 21, 31, 33, 40, 84, 23, 57, 35, 61, 32, 56, 63, 82, 88, 49,
       55,  8, 75, 34, 14, 29,  9,  1, 13, 72, 89, 74, 79, 58, 59, 53, 91,
       54, 52, 20, 78, 19, 39, 18, 80,  0])

Här är ett exempel på vad som händer när man tar bort en kolumn:

In [66]:
columns_1 = np.delete(columns,4)

In [67]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])

In [68]:
order_1 = get_order(columns_1)

In [69]:
x = np.array_equal(order_0, order_1)

In [70]:
x

False

In [71]:
columns_1

array([ 2,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18])

Om vi tar en uppsättning kolumner behöver vi loopa igenom alla uppsättningar som saknar en kolumn. Något i den här stilen:

In [72]:
size = columns.size
print(size)
a = np.empty([size,size-1],dtype=int)
#print(a)
for index, item in enumerate(columns):
    a[index]=np.delete(columns,index)
    print(a[index])

15
[ 4  5  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  5  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  6  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  7  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  8  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  9 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8 10 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 11 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 12 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 13 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 14 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 16 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 17 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 16 18]
[ 2  4  5  6  7  8  9 10 11 12 13 14 16 17]


Om vi ska göra en funktion av det hela och få ut en array av de överflödiga kolumnerna:

In [91]:
def remove_dimensions (columns):
    columns_1 = np.empty([columns.size,columns.size-1],dtype=int)
    superfluous = []
    #print(a)
    for index, item in enumerate(columns):
        columns_1[index]=np.delete(columns,index)
        if np.array_equal(get_order(columns), get_order(columns_1[index])):
            superfluous.append(item)
    return superfluous

In [93]:
print (columns_0)
superfluous = remove_dimensions (columns_0)
superfluous

[ 2  4  5  6  7  8  9 10 11 12 13 14 16 17 18]


[9, 16, 17, 18]